In [29]:
%run runaway_functionsv3
%matplotlib qt
cl = Cluster('LP_1487')
# cld = ClusterDias(cl.name)
# plot_cmd(cl, isochrones=[Isochrone(cld)])
# cl.plot_traceback_clean()

In [17]:
import os
from astropy.table import Table
import astropy.io.ascii
import numpy as np

def latex_text(cluster):
    cld = ClusterDias(cluster.name)
    ra, dec = cluster.skycoord.ra, cluster.skycoord.dec
    ra_str, dec_str = ra.to_string(format='latex')[1:-1], dec.to_string(format='latex')[1:-1]
    dist_str = str(round(cluster.distance.value,1)) + r"\pm" + str(round(cluster.e_distance.value,1))
    
    intro_text = rf"""\subsection{{Introduction}}
    {cluster.name.replace('_', ' ')} is located at $\alpha = {ra_str}, \delta = {dec_str}$ at a distance of ${dist_str}$ pc.
    {cluster.N} out of the {cld.N} mentioned in Dias have been selected as members. Using these, the kinematic parameters of the cluster have been determined. These are compared to the kinematic parameters of the runaway star in \ref{{tab:{cluster.name}_kinematics}}.
    """
    
    # Merged latex_table_kinematics function
    t_dict = {
        r'Object':[cluster.name.replace('_',' ')],
        r'$\alpha (^\circ)$': [f"{(cluster.skycoord.ra):.2f}".replace(' deg','')],
        r'$\delta (^\circ)$': [f"{(cluster.skycoord.dec):.2f}".replace(' deg','')],
        "Radius. (')":[round(cluster.r50.value*60,2)],
        "N":[cluster.N],
        r"$\mu_{\alpha}^*$ (mas/yr)":[f"{cluster.all['pmRA']:.2f}"+"$\pm$"+f"{cluster.all['e_pmRA']:.2f}"],
        r"$\mu_{\delta}$ (mas/yr)":[f"{cluster.all['pmDE']:.2f}"+"$\pm$"+f"{cluster.all['e_pmDE']:.2f}"],
        r"$d$ (pc)":[f"{cluster.all['Dist']:.0f}"+"$\pm$"+f"{cluster.all['e_Dist']:.0f}"],
    }
    
    latexdict = {
        'tabletype': 'table*',
        'tablealign': 'h',
        'header_start': r'\hline',
        'data_start': r'\hline',
        'data_end': r'\hline', 
        'caption': f'Kinematic parameters of {cluster.name.replace("_"," ")}',
        'preamble': r'\label{tab:' + f'{cluster.name}_kinematics' + '}'
    }
    
    astropy_table = Table(t_dict)
    output_filename = 'text.txt'
    
    astropy.io.ascii.write(astropy_table, format='latex', output=output_filename, overwrite=True, latexdict=latexdict)
    
    with open(output_filename, 'r+') as f:
        lines = f.readlines()
        lines[1], lines[2] = lines[2], lines[1]
        f.seek(0)
        f.writelines(lines)
        table_text = ''.join(lines)
    
    os.remove(output_filename)
    
    # Combine intro_text and table_text
    full_text = intro_text + "\n\n" + table_text
    
    filename = f"{cluster.name}.tex"
    
    with open(filename, 'w') as file:
        file.write(full_text)
    
    return full_text

# Example usage
print(latex_text(cl))


\subsection{Introduction}
    LP 1487 is located at $\alpha = 160^\circ29{}^\prime32.64{}^{\prime\prime}, \delta = -58^\circ22{}^\prime22.44{}^{\prime\prime}$ at a distance of $2710.0\pm177.0$ pc.
    29 out of the 195 mentioned in Dias have been selected as members. Using these, the kinematic parameters of the cluster have been determined. These are compared to the kinematic parameters of the runaway star in \ref{tab:LP_1487_kinematics}.
    

\begin{table*}[h]
\caption{Kinematic parameters of LP 1487}
\label{tab:LP_1487_kinematics}
\begin{tabular}{cccccccc}
\hline
Object & $\alpha (^\circ)$ & $\delta (^\circ)$ & Radius. (') & N & $\mu_{\alpha}^*$ (mas/yr) & $\mu_{\delta}$ (mas/yr) & $d$ (pc) \\
\hline
LP 1487 & 160.49 & -58.37 & 5.04 & 29 & -6.56$\pm$0.19 & 3.34$\pm$0.19 & 2710$\pm$177 \\
\hline
\end{tabular}
\end{table*}



In [18]:
def latex_text(cluster, n_members=None):
    cld = ClusterDias(cluster.name)
    ra, dec = cluster.skycoord.ra, cluster.skycoord.dec
    ra_str, dec_str = ra.to_string(format='latex')[1:-1], dec.to_string(format='latex')[1:-1]
    dist_str = str(round(cluster.distance.value,1)) + r"\pm" + str(round(cluster.e_distance.value,1))
    
    intro_text = rf"""\subsection{{Introduction}}
    {cluster.name.replace('_', ' ')} is located at $\alpha = {ra_str}, \delta = {dec_str}$ at a distance of ${dist_str}$ pc.
    {cluster.N} out of the {cld.N} mentioned in Dias have been selected as members. Using these, the kinematic parameters of the cluster have been determined. These are compared to the kinematic parameters of the runaway star in \ref{{tab:{cluster.name}_kinematics}}. Some of the brightest memebrs fo the cluster are shown in table \ref{{tab:{cluster.name}_members}}.
    
    \subsection{{CMD and proper motion}}
    
    
    """
    
    # Kinematics table generation
    t_dict_kinematics = {
        r'Object': [cluster.name.replace('_', ' ')],
        r'$\alpha (^\circ)$': [f"{(cluster.skycoord.ra):.2f}".replace(' deg', '')],
        r'$\delta (^\circ)$': [f"{(cluster.skycoord.dec):.2f}".replace(' deg', '')],
        r"$d$ (pc)": [f"{cluster.all['Dist']:.0f}" + "$\pm$" + f"{cluster.all['e_Dist']:.0f}"],
        r"$\mu_{\alpha}^*$ (mas/yr)": [f"{cluster.all['pmRA']:.2f}" + "$\pm$" + f"{cluster.all['e_pmRA']:.2f}"],
        r"$\mu_{\delta}$ (mas/yr)": [f"{cluster.all['pmDE']:.2f}" + "$\pm$" + f"{cluster.all['e_pmDE']:.2f}"],
        r"$v_R$": [f"{cluster.all['RV']:.2f}" + "$\pm$" + f"{cluster.all['e_RV']:.2f}" if not np.ma.is_masked(cluster.all['RV']) else "N/A"] ,
        "N": [cluster.N],
        "Radius. (')": [round(cluster.r50.value * 60, 2)],
    }
    
    latexdict_kinematics = {
        'tabletype': 'table*',
        'tablealign': 'h',
        'header_start': r'\hline',
        'data_start': r'\hline',
        'data_end': r'\hline', 
        'caption': f'Kinematic parameters of {cluster.name.replace("_"," ")}',
        'preamble': r'\label{tab:' + f'{cluster.name}_kinematics' + '}'
    }
    
    astropy_table_kinematics = Table(t_dict_kinematics)
    kinematics_output_filename = 'kinematics_text.txt'
    
    astropy.io.ascii.write(astropy_table_kinematics, format='latex', output=kinematics_output_filename, overwrite=True, latexdict=latexdict_kinematics)
    
    with open(kinematics_output_filename, 'r+') as f:
        lines = f.readlines()
        lines[1], lines[2] = lines[2], lines[1]
        f.seek(0)
        f.writelines(lines)
        kinematics_table_text = ''.join(lines)
    
    os.remove(kinematics_output_filename)
    
    # Members table generation
    table = cluster.mymembers
    table.sort('Gmag')
    table = table[:n_members]
    
    formatted_distances = []
    formatted_parallaxes = []
    formatted_gmags = []
    formatted_bprps = []
    
    for row in table:
        distance = row['rgeo']
        plx = row['Plx']
        e_plx = row['e_Plx']
        upper_error = row['B_rgeo'] - row['rgeo']
        lower_error = -row['b_rgeo'] + row['rgeo']
        gmag = row['Gmag']
        e_gmag = row['e_Gmag']
        bprp = row['BP-RP']
        e_bprp = row['e_BP-RP']
        
        formatted_distance = f"${distance:.0f}^{{+{upper_error:.0f}}}_{{-{lower_error:.0f}}}$"
        formatted_parallax = f"${plx:.4f}\pm{e_plx:.4f}$"
        formatted_gmag = f"${gmag:.3f}\pm{e_gmag:.3f}$"
        formatted_bprp = f"${bprp:.3f}\pm{e_bprp:.3f}$"
        
        formatted_distances.append(formatted_distance)
        formatted_parallaxes.append(formatted_parallax)
        formatted_gmags.append(formatted_gmag)
        formatted_bprps.append(formatted_bprp)

    table['formatted_distance'] = formatted_distances
    table['formatted_parallax'] = formatted_parallaxes
    table['formatted_gmag'] = formatted_gmags
    table['formatted_bprp'] = formatted_bprps
    
    t_dict_members = {
        'Gaia DR3 Source': table['Source'],
        r"$r_{\text{geo}}$ (pc)": table['formatted_distance'],
        r"$\pi$ (mas)": table['formatted_parallax'],
        r"$G$ (mag)": table['formatted_gmag'],
        r"$G_{\text{BP}}-G_{\text{RP}}$ (mag)": table['formatted_bprp']
    }
    
    latexdict_members = {
        'tabletype': 'table*',
        'tablealign': 'h',
        'header_start': r'\hline',
        'data_start': r'\hline',
        'data_end': r'\hline', 
        'caption': f'Selected members of {cluster.name.replace("_"," ")}',
        'preamble': r'\label{tab:' + f'{cluster.name}_members' + '}'
    }
    
    astropy_table_members = Table(t_dict_members)
    members_output_filename = 'members_text.txt'
    
    astropy.io.ascii.write(astropy_table_members, format='latex', output=members_output_filename, overwrite=True, latexdict=latexdict_members)
    
    with open(members_output_filename, 'r+') as f:
        lines = f.readlines()
        lines[1], lines[2] = lines[2], lines[1]
        f.seek(0)
        f.writelines(lines)
        members_table_text = ''.join(lines)
    
    os.remove(members_output_filename)
    
    # Combine intro_text, kinematics_table_text, and members_table_text
    full_text = intro_text + "\n\n" + kinematics_table_text + "\n\n" + members_table_text
    
    filename = f"{cluster.name}.tex"
    
    with open(filename, 'w') as file:
        file.write(full_text)
    
    return full_text

# Example usage
print(latex_text(cl, n_members=10))


\subsection{Introduction}
    LP 1487 is located at $\alpha = 160^\circ29{}^\prime32.64{}^{\prime\prime}, \delta = -58^\circ22{}^\prime22.44{}^{\prime\prime}$ at a distance of $2710.0\pm177.0$ pc.
    29 out of the 195 mentioned in Dias have been selected as members. Using these, the kinematic parameters of the cluster have been determined. These are compared to the kinematic parameters of the runaway star in \ref{tab:LP_1487_kinematics}. Some of the brightest memebrs fo the cluster are shown in table \ref{tab:LP_1487_members}.
    
    \subsection{CMD and proper motion}
    
    
    

\begin{table*}[h]
\caption{Kinematic parameters of LP 1487}
\label{tab:LP_1487_kinematics}
\begin{tabular}{ccccccccc}
\hline
Object & $\alpha (^\circ)$ & $\delta (^\circ)$ & $d$ (pc) & $\mu_{\alpha}^*$ (mas/yr) & $\mu_{\delta}$ (mas/yr) & $v_R$ & N & Radius. (') \\
\hline
LP 1487 & 160.49 & -58.37 & 2710$\pm$177 & -6.56$\pm$0.19 & 3.34$\pm$0.19 & N/A & 29 & 5.04 \\
\hline
\end{tabular}
\end{table*}


\b

In [88]:
cl.runaways()

Name,RA_ICRS_1,DE_ICRS_1,rgeo,Teff,Temp. Est,v_pec,e_v_pec,v_pec3d,e_v_pec3d,HIP,TYC2,Source,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Gmag,BP-RP,BPmag,RPmag,b_rgeo,B_rgeo,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,e_rmRA,rmDE,e_rmDE,logg,RV,e_RV,rRV,e_rRV,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6
,deg,deg,pc,K,K,km / s,km / s,km / s,km / s,,,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,mag,mag,mag,mag,pc,pc,,,,,"deg,deg,pc",mas / yr,mas / yr,mas / yr,mas / yr,log(cm.s**-2),km / s,km / s,km / s,km / s,,,,,,,,
str28,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,str11,int64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,SkyCoord,float64,float32,float64,float32,float64,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,str1,str1
Gaia DR3 5350715479093241728,160.52828415935,-58.42891748508,2716.38672000,9419.2,14550.0,20.58776702309437,4.396459900210389,--,--,--,--,5350715479093241728,0.3337,0.0183,-5.162,2.565,0.019,0.018,0.909,14.661097,0.637711,14.896277,14.258567,2595.32788000,2836.99316000,0.002775,0.003056,0.003853,0.006909,"160.52828415935,-58.42891748508,2716.38672",1.400,0.214,-0.772,0.213,3.6995,--,--,--,--,25734.05483,7.783,1.503e+04,17.24,1.569e+04,10.84,--,--
Gaia DR3 5350727367562696576,160.43756098606,-58.39531882342,2819.23364000,--,13630.0,23.22132431885084,4.941029207922377,--,--,--,--,5350727367562696576,0.3188,0.0175,-8.078,2.487,0.018,0.018,1.366,13.798375,0.659433,14.043551,13.384117,2694.43945000,2954.89307000,0.002764,0.002862,0.003843,0.006705,"160.43756098606,-58.39531882342,2819.23364",-1.516,0.213,-0.850,0.213,--,--,--,--,--,56963.50361,11.65,3.296e+04,19.38,3.512e+04,22.44,--,--
Gaia DR3 5350727470642189696,160.42392612814,-58.40127836045,2535.07227000,--,11710.0,21.24732741193923,4.54224987192842,--,--,--,--,5350727470642189696,0.3579,0.0191,-5.901,1.697,0.019,0.019,1.036,14.746082,0.720582,15.022784,14.302202,2421.79395000,2624.90283000,0.002767,0.002973,0.003898,0.006871,"160.42392612814,-58.40127836045,2535.07227",0.661,0.214,-1.640,0.214,--,--,--,--,--,23796.54314,5.675,1.338e+04,12.63,1.508e+04,13.28,--,--


In [101]:
cl = Cluster('Berkeley_97')
cluster_part = Table(cl.all['Cluster', 'RA_ICRS', 'DE_ICRS','Dist','e_Dist', 'pmRA','e_pmRA','pmDE','e_pmDE','RV','e_RV'])
runaway_part = cl.runaways()['Source', 'RA_ICRS_1', 'DE_ICRS_1','rgeo','B_rgeo','b_rgeo', 'pmRA','e_pmRA','pmDE','e_pmDE','RV','e_RV']
runaway_part['Source'] = runaway_part['Source'].astype(str)
runaway_part['e_rgeo'] = runaway_part['B_rgeo']-runaway_part['b_rgeo']
runaway_part.rename_column('RA_ICRS_1','RA_ICRS')
runaway_part.rename_column('DE_ICRS_1','DE_ICRS')
runaway_part.rename_column('rgeo','Dist')
runaway_part.rename_column('e_rgeo','e_Dist')
runaway_part.rename_column('Source','Cluster')
runaway_part.remove_columns(['B_rgeo', 'b_rgeo'])

cluster_part['Cluster'][0] = cl.name.replace('_',' ')

import numpy as np

# Assuming cluster_part and runaway_part are already defined as in your script
kc = vstack([cluster_part, runaway_part])

# Define a function to format values and errors
def format_with_error(value, error, precision=0):
    if np.ma.is_masked(value) or np.ma.is_masked(error):
        return '-'  # Return a placeholder for masked values
    try:
        rounded_value = round(float(value), precision)
        rounded_error = round(float(error), precision)
        return f"{rounded_value} ± {rounded_error}"
    except (TypeError, ValueError):
        return '-'  # Return a placeholder for values that cannot be converted

# Define a function to round RA and DEC values
def format_coordinates(value, precision=3):
    if np.ma.is_masked(value):
        return '-'
    try:
        rounded_value = round(float(value), precision)
        return f"{rounded_value}"
    except (TypeError, ValueError):
        return '-'

# Create a new Table for LaTeX names
kc_latex = Table()

# Mapping old names to LaTeX names
name_mapping = {
    'Cluster': r'Cluster',
    'RA_ICRS': r'$\alpha$',
    'DE_ICRS': r'$\delta$',
    'Dist': r"$d$",
    'pmRA': r"$\mu_{\alpha}^*$",
    'pmDE': r"$\mu_{\delta}$",
    'RV': r"$v_R$"
}

# Manually handle each column's precision and formatting

# Format and round the 'Object' column
kc_latex[name_mapping['Cluster']] = kc['Cluster']

# Format and round the RA (α) values
kc_latex[name_mapping['RA_ICRS']] = [
    format_coordinates(value, precision=3) for value in kc['RA_ICRS']
]

# Format and round the DEC (δ) values
kc_latex[name_mapping['DE_ICRS']] = [
    format_coordinates(value, precision=3) for value in kc['DE_ICRS']
]
# Format and round the 'Dist' (d) values with their errors as integers
kc_latex[name_mapping['Dist']] = [
    f"{int(round(value))} ± {int(round(error))}"
    for value, error in zip(kc['Dist'], kc['e_Dist'])
]

# Format and round the 'pmRA' (μ_α*) values with their errors
kc_latex[name_mapping['pmRA']] = [
    format_with_error(value, error, precision=1)
    for value, error in zip(kc['pmRA'], kc['e_pmRA'])
]

# Format and round the 'pmDE' (μ_δ) values with their errors
kc_latex[name_mapping['pmDE']] = [
    format_with_error(value, error, precision=1)
    for value, error in zip(kc['pmDE'], kc['e_pmDE'])
]

# Format and round the 'RV' (v_R) values with their errors
kc_latex[name_mapping['RV']] = [
    format_with_error(value, error, precision=1)
    for value, error in zip(kc['RV'], kc['e_RV'])
]

# Print the LaTeX table or save it as needed
# print(kc_latex)

# Optionally save to a LaTeX file
ascii.write(kc_latex, format="latex", 
            latexdict={
                    'tabletype':'table*','tablealign':'h',
                    'header_start':r'\hline','data_start':r'\hline','data_end': r'\hline', 
                    'caption':f'Kinematic parameters of {cl.name.replace("_"," ")} compared to the runaways' if len(cl.runaways())>1 else f'Kinematic parameters of {cl.name.replace("_"," ")} compared to the runaway',
                    'preamble':'\label{tab:'+f'{cl.name}_kinematics_runaways'+'}',
                    
                       'units': {
                                 r'Cluster':'or GDR3 Source',
                                 r'$\alpha$': '(deg)', 
                                 r'$\delta$': '(deg)',
                                 r"$d$":'(pc)',
                                 r"$\mu_{\alpha}^*$": '(mas/yr)',
                                 r"$\mu_{\delta}$": '(mas/yr)',
                                 r"$v_R$":'(km/s)'
                                 }})



\begin{table*}[h]
\label{tab:Berkeley_97_kinematics_runaways}
\caption{Kinematic parameters of Berkeley 97 compared to the runaway}
\begin{tabular}{ccccccc}
\hline
Cluster & $\alpha$ & $\delta$ & $d$ & $\mu_{\alpha}^*$ & $\mu_{\delta}$ & $v_R$ \\
or GDR3 Source & (deg) & (deg) & (pc) & (mas/yr) & (mas/yr) & (km/s) \\
\hline
Berkeley 97 & 339.85 & 59.005 & 2769 ± 132 & -2.7 ± 0.1 & -1.9 ± 0.1 & - \\
2008384504995669248 & 339.896 & 58.995 & 2620 ± 298 & -0.4 ± 0.0 & -1.4 ± 0.0 & - \\
\hline
\end{tabular}
\end{table*}


In [39]:
from astropy.io import ascii
data = {'name': ['bike', 'car'], 'mass': [75,1200], 'speed': [10, 130]}
t = ascii.write(kc, format="latex",
            # latexdict={'units': {'mass': 'kg', 'speed': 'km/h'}},
            )


\begin{table}
\begin{tabular}{ccccccccccc}
Object & RA_ICRS & DE_ICRS & Dist & e_Dist & pmRA & e_pmRA & pmDE & e_pmDE & RV & e_RV \\
 & $\mathrm{{}^{\circ}}$ & $\mathrm{{}^{\circ}}$ & $\mathrm{pc}$ & $\mathrm{pc}$ & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{km\,s^{-1}}$ & $\mathrm{km\,s^{-1}}$ \\
LP_1487 & 160.49240000000 & -58.37290000000 & 2710.00000000 & 177.00000000 & -6.562 & 0.195 & 3.337 & 0.195 &  &  \\
Gaia DR3 5350715479093241728 & 160.52828415935 & -58.42891748508 & 2716.38672000 & 241.66528000 & -5.162 & 0.019 & 2.565 & 0.018 &  &  \\
Gaia DR3 5350727367562696576 & 160.43756098606 & -58.39531882342 & 2819.23364000 & 260.45362000 & -8.078 & 0.018 & 2.487 & 0.018 &  &  \\
Gaia DR3 5350727470642189696 & 160.42392612814 & -58.40127836045 & 2535.07227000 & 203.10888000 & -5.901 & 0.019 & 1.697 & 0.019 &  &  \\
\end{tabular}
\end{table}


In [30]:
from astropy.table import Table, vstack, Column
import astropy.io.ascii
import numpy as np
import os

# Define function to format columns with error
def format_with_error(values, errors):
    return [f"{value:.2f}$\pm${error:.2f}" if not np.ma.is_masked(value) else "N/A" for value, error in zip(values, errors)]

# Combine and format tables
def combine_and_format_tables(cluster, runaway_table):
    warnings.filterwarnings("ignore", category=MergeConflictWarning)
    
    # Extract and rename columns in the cluster table
    cluster_part = Table(cluster.all['Cluster', 'RA_ICRS', 'DE_ICRS', 'Dist', 'e_Dist', 'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE', 'RV', 'e_RV'])
    cluster_part.rename_column('Cluster', 'Object')
    
    # Extract and process runaways table
    runaway_part = runaway_table[['Name', 'RA_ICRS_1', 'DE_ICRS_1', 'rgeo', 'B_rgeo', 'b_rgeo', 'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE', 'RV', 'e_RV']]
    runaway_part.rename_column('Name', 'Object')
    runaway_part.rename_column('RA_ICRS_1', 'RA_ICRS')
    runaway_part.rename_column('DE_ICRS_1', 'DE_ICRS')
    runaway_part.rename_column('rgeo', 'Dist')
    runaway_part['e_Dist'] = runaway_part['B_rgeo'] - runaway_part['b_rgeo']
    runaway_part.rename_column('e_Dist', 'e_Dist')
    runaway_part.remove_columns(['B_rgeo', 'b_rgeo'])
    
    # Ensure 'Object' column is of string type for both tables
    cluster_part['Object'] = cluster_part['Object'].astype(str)
    runaway_part['Object'] = runaway_part['Object'].astype(str)

    # Combine the two tables
    combined_table = vstack([cluster_part, runaway_part])

    # Define LaTeX formatting for the combined table
    latex_dict = {
        'tabletype': 'table*',
        'tablealign': 'h',
        'header_start': r'\hline',
        'data_start': r'\hline',
        'data_end': r'\hline',
        'caption': f'Combined Kinematic Parameters of Cluster and Runaway Stars',
        'preamble': r'\label{tab:combined_kinematics}'
    }
    
    # Export the table to LaTeX format
    output_filename = 'combined_table_text.txt'
    astropy.io.ascii.write(combined_table, format='latex', output=output_filename, overwrite=True, latexdict=latex_dict)
    
    # Read the LaTeX output file, swap headers, and prepare final text
    with open(output_filename, 'r+') as f:
        lines = f.readlines()
        lines[1], lines[2] = lines[2], lines[1]  # Swap the header lines
        f.seek(0)
        f.writelines(lines)
        combined_table_text = ''.join(lines)

    # Remove the temporary file
    os.remove(output_filename)
    
    return combined_table_text

# Example usage
# Assuming `cl` is your cluster object and `cl.runaways()` returns the runaways table
combined_latex_text = combine_and_format_tables(cl, cl.runaways())
print(combined_latex_text)


\begin{table*}[h]
\caption{Combined Kinematic Parameters of Cluster and Runaway Stars}
\label{tab:combined_kinematics}
\begin{tabular}{ccccccccccc}
\hline
Object & RA_ICRS & DE_ICRS & Dist & e_Dist & pmRA & e_pmRA & pmDE & e_pmDE & RV & e_RV \\
 & $\mathrm{{}^{\circ}}$ & $\mathrm{{}^{\circ}}$ & $\mathrm{pc}$ & $\mathrm{pc}$ & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{km\,s^{-1}}$ & $\mathrm{km\,s^{-1}}$ \\
\hline
LP_1487 & 160.49240000000 & -58.37290000000 & 2710.00000000 & 177.00000000 & -6.562 & 0.195 & 3.337 & 0.195 &  &  \\
Gaia DR3 5350715479093241728 & 160.52828415935 & -58.42891748508 & 2716.38672000 & 241.66528000 & -5.162 & 0.019 & 2.565 & 0.018 &  &  \\
Gaia DR3 5350727367562696576 & 160.43756098606 & -58.39531882342 & 2819.23364000 & 260.45362000 & -8.078 & 0.018 & 2.487 & 0.018 &  &  \\
Gaia DR3 5350727470642189696 & 160.42392612814 & -58.40127836045 & 2535.07227000 & 203.10888000 & -5.901 & 0.019 & 1.69